In [1]:
import dataProsessing as dp
import numpy as np
import os
import io
import json
import math, random, time
from numpy import linalg as LA
import music21  
from tabulate import tabulate
import collections as coll

Because I have no real idea how to handle filesystems and importing across a branched structure everything is in scripts. I do not apologise.
Furthermore since jupyter notebooks run from their own directory (in this case, Scripts) we run os.chdir("../") to get to the projects root directory (corpus-of-chords). Running this call multiple times without resetting the kernal will continue to crawl up the tree, so do not do that.

In [2]:
#RUN THIS ONCE
os.chdir("../")

This is the k_means algorithm written in Chandler's class MAT167 HW4 summer 2022. We will run it on our weights output from word2vec, then view information about the various clusters.

In [3]:
def k_means(M,k,tol = .01, max_iters = 100):
    ################################## your code goes here ##########################################

    #inits
    steps = 0
    prev = tol+1
    current = 0
    m = len(M)
    n = len(M[0,:])
    initialize = random.sample(range(n),k)
    KM = M[:,initialize].astype(np.float32) 
    partition = [0] * n
    
    while (abs(prev-current) > tol) and (steps < max_iters):
        steps += 1
        for i in range(n):
            # find closest column of KM
            prev_diff = -1
            for j in range(k):
                diff = LA.norm(M[:,i]-KM[:,j],2)
                if prev_diff <0 or diff<prev_diff:
                    prev_diff = diff
                    partition[i] = j
        
        for j in range(k):
            # KM[:,j] = take mean of cols such that partition[s] == j
            s_indices = [ s for s in range(n) if partition[s] == j]
            S = M[:,s_indices]
            KM[:,j] = S.sum(axis = 1) / len(s_indices)
        
        prev = current
        current = sum([LA.norm(M[:,i]-KM[:,partition[i]]) for i in range(n)])
        
        print(f'after {steps} steps the residue is {current}')
        
    return(KM, partition)


Load corpi of chords:

Input needed: set model_name manually from available files in weights folder.
- vocab: dictionary with key:value as (chord vector in midi format):index
- inv_vocab: chords as vector using midi numbers
- weights: weight vectors outputted from word2vec

inv_vocab and weights indexing are such that the ith entry in inv_vocab corresponds with the ith entry in weights

In [4]:
#set model_name manually
model_name = "d_128 n_15"

datasetName = "Jsb16thSeparated(t_60_rr_re)"

#add option for / vs \ check TODO
vocab = dataset = json.load(open("Data/Corpi/" + datasetName + "/" + datasetName +"_vocab.json"))
inv_vocab = vocab["inv_vocab"]
vocab = vocab["vocab"]
weights = json.load(open("Data/Corpi/" + datasetName + "/weights/" + model_name))
weights_km = np.array(weights).T

Gather chord information and format ...

- labels_km: [ [chord name, chord root tone, chord quality] ] as determined by music21. 

In [5]:
labels_km = dp.getChordLabels(inv_vocab,True,True) #gets (chord name, chord root, chord quality)

0th step
300th step
600th step
900th step
1200th step
1500th step
1800th step
2100th step
2400th step
2700th step
3000th step
3300th step
3600th step
3900th step
4200th step
4500th step
4800th step
5100th step
5400th step
5700th step
6000th step
6300th step
6600th step
6900th step


Now we run k_means! Here we use 24 clusters.

In [6]:
k = 24
KM,partition = k_means(weights_km,k,.001,100)

after 1 steps the residue is 14787.943567769675
after 2 steps the residue is 14206.325539537305
after 3 steps the residue is 14049.04831444525
after 4 steps the residue is 13999.48478580884
after 5 steps the residue is 13976.67267949948
after 6 steps the residue is 13959.491910052968
after 7 steps the residue is 13948.905981412672
after 8 steps the residue is 13940.848643981913
after 9 steps the residue is 13934.177114017504
after 10 steps the residue is 13928.373365097888
after 11 steps the residue is 13922.722951906353
after 12 steps the residue is 13917.36369683208
after 13 steps the residue is 13912.11743229594
after 14 steps the residue is 13907.176969093482
after 15 steps the residue is 13901.818610377308
after 16 steps the residue is 13897.418531176785
after 17 steps the residue is 13894.35485566651
after 18 steps the residue is 13892.438833658452
after 19 steps the residue is 13891.22305146804
after 20 steps the residue is 13890.533280455084
after 21 steps the residue is 13890.

Now we print a table containing some information about the clustering. Currently it is set to list the partitions with the top n most frequent chords. See the text files in OutputAnalysis folder for more examples and analysis.


In [9]:
namefreqs,rootfreqs,qualfreqs,partsizes =  dp.partitionFreqs(partition,labels_km,weights,k)
title,tabledata,headers = dp.tabulate_partitions("ChordNames",namefreqs,partsizes,5,k) 
print(title)
print(tabulate(tabledata,headers))

Partitions, ChordNames
  Partition    part. size  #1 most common              #2 most common                                #3 most common                                #4 most common                               #5 most common
-----------  ------------  --------------------------  --------------------------------------------  --------------------------------------------  -------------------------------------------  ----------------------------------------------
          0        0.0288  ['Eb-major triad', 0.0394]  ['Bb-major triad', 0.0296]                    ['C#-major triad', 0.0246]                    ['Bb-dominant seventh chord', 0.0246]        ['Bb-minor triad', 0.0246]
          1        0.0904  ['F-major triad', 0.0298]   ['Eb-major triad', 0.0235]                    ['C-minor triad', 0.0204]                     ['G-major triad', 0.0173]                    ['A-half-diminished seventh chord', 0.0157]
          2        0.0433  ['C-major triad', 0.0361]   ['C-major seventh cho